In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
pd.options.display.max_columns=None
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from dateutil.relativedelta import *
from dateutil.rrule import *
from datetime import datetime as dt

%config InteractiveShell.ast_node_interactivity = 'all'
#import dependencies

In [2]:
df = pd.read_parquet("../Proyecto-Taxis-NYC/data/yellow_t_2018-01.parquet", engine="pyarrow")

In [3]:
def t_datime(df):
    df["dayofmonth"] = df['tpep_pickup_datetime'].apply(pd.to_datetime).dt.day
    df["hora"]= df.tpep_pickup_datetime.dt.floor("H")

    df.drop(columns=["congestion_surcharge","airport_fee","store_and_fwd_flag"], inplace=True)
    return df

def t_int(df):
    df['passenger_count'] = pd.to_numeric(df['passenger_count'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    df['PULocationID'] = pd.to_numeric(df['PULocationID'], downcast="integer" )
    df['DOLocationID'] = pd.to_numeric(df['DOLocationID'], downcast="integer" )
    df['payment_type'] = pd.to_numeric(df['payment_type'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    df['RatecodeID'] = pd.to_numeric(df['RatecodeID'], downcast="integer" )
    return df

def t_float(df):
    df['trip_distance'] = pd.to_numeric(df['trip_distance'], downcast="float" )
    df['fare_amount'] = pd.to_numeric(df['fare_amount'], downcast="float" )
    df['extra'] = pd.to_numeric(df['extra'], downcast="float" )
    df['mta_tax'] = pd.to_numeric(df['mta_tax'], downcast="float" )
    df['tip_amount'] = pd.to_numeric(df['tip_amount'], downcast="float" )
    df['tolls_amount'] = pd.to_numeric(df['tolls_amount'], downcast="float" )
    df['improvement_surcharge'] = pd.to_numeric(df['improvement_surcharge'], downcast="float" )
    df['total_amount'] = pd.to_numeric(round(df['total_amount'] ))
    return df

def total_transform(df):
    df = t_datime(df)
    df = t_int(df)
    df = t_float(df)
    return df

df =  total_transform(df)

In [4]:
df_Impar = df[df.dayofmonth % 3 == 0]


In [5]:
def sorted(df):
    global bol
    bol = 1
    df.sort_values(by='tpep_pickup_datetime', ascending=True, inplace=True)
    df = df[df["tpep_pickup_datetime"] >= '2018-01-01 00:00:00']
    if bol == 1:
        df = df[df["tpep_pickup_datetime"] <= '2018-02-01 00:00:00']
        bol = 0
    return df

df_Impar = sorted(df_Impar)

In [6]:
def boroughs (df_Impar):
    #se carga el csv con las zonas
    zona = pd.read_csv("taxi+_zone_lookup.csv")
    #se borran las columnas que no usamos
    zona.drop(columns=["Zone","service_zone"], inplace=True)
    #cambiamos el nombre para poder hacer el merge
    zona.rename(columns={"LocationID": "PULocationID"},inplace=True)
    #se hace el merge y se crea la columnna resultante del merge "Borough"
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["PULocationID"])
    #se le cambia el nombre a "boroug" para que haga referencia al borough de partida
    df_Impar.rename(columns={"Borough": "PUBorough"},inplace=True)
    #mismo procedimiento para borough de llegada
    zona.rename(columns={"PULocationID": "DOLocationID"},inplace=True)
    df_Impar = pd.merge(df_Impar,zona, how="left", on=["DOLocationID"])
    df_Impar.rename(columns={"Borough": "DOLocation"},inplace=True)
    return df_Impar

df_Impar = boroughs(df_Impar)

In [7]:
#documentacion mismo nombre del clima en cada update

clima = pd.read_csv("C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/clima.csv")
clima.drop(columns=["name","dew","humidity","precipprob","preciptype","severerisk","uvindex","icon","stations"], inplace=True)
clima.drop(columns=["windgust","windspeed","winddir","sealevelpressure","cloudcover","visibility","solarradiation","solarenergy"], inplace=True)
clima["datetime"]=[x.replace('T'," ") for x in clima["datetime"]]
clima.datetime = clima['datetime'].apply(pd.to_datetime)

In [8]:
def t_clima():
    clima['temp'] = pd.to_numeric(clima['temp'], downcast="float" )
    clima['feelslike'] = pd.to_numeric(round(clima['feelslike'] ))
    clima['precip'] = pd.to_numeric(clima['precip'], downcast="float" )
    clima['snow'] = pd.to_numeric(clima['snow'], downcast="float" )
    clima['snowdepth'] = pd.to_numeric(clima['snowdepth'], downcast="float" )
    clima["conditions"]=clima["conditions"].astype(str)
    return clima

clima = t_clima()

In [9]:
def control_clima(dat,clima):
    #aplicar sort fecha
    clima.sort_values(by='datetime', ascending=True, inplace=True)
    date = dat
    if (clima["datetime"] >= date).any():
        clima = clima[clima["datetime"] >= date]
        date_next= date + relativedelta(months=+1)
        clima = clima[clima["datetime"] <= date_next]
        return clima
    else:
        date+= relativedelta(months=+1)
        return control_clima(dat)

clima = control_clima(dt(2018,1,1),clima)

In [10]:
#clima in global

def aux ():
    
    clima.rename(columns={"datetime": "hora"},inplace=True)
    return clima

clima = aux()

In [11]:
df_Impar = pd.merge(df_Impar,clima, how="left", on=["hora"])

borro los datos en cero

In [12]:
def neg_values(df_Impar):   
    df_Impar = df_Impar.drop(df_Impar[df_Impar.fare_amount<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.trip_distance<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.total_amount<0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.RatecodeID==6].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.trip_distance==0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.fare_amount==0].index)
    df_Impar = df_Impar.drop(df_Impar[df_Impar.total_amount==0].index)
    return df_Impar

df_Impar=neg_values(df_Impar)

marco los outliers.

In [13]:
#Tratamiento de Outliers columna trip_distance
Q1 = df_Impar["trip_distance"].quantile(0.01)
Q3 = df_Impar["trip_distance"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_Sup_trip = (Q3 + (1.5*IQR)) 
#se crea la columna marcador
df_Impar["trip_distance_out"] = df_Impar.apply(lambda row: 1 if row["trip_distance"]>outliers_Sup_trip else 0, axis=1)

#se marco trip distance, ahora vamos por fare amount

#Tratamiento de Outliers columna fare_amount
Q1 = df_Impar["fare_amount"].quantile(0.01)
Q3 = df_Impar["fare_amount"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_Sup_fare = (Q3 + (1.5*IQR)) 
#se marco fare amount
df_Impar["fare_amount_out"] = df_Impar.apply(lambda row: 1 if row["fare_amount"]>outliers_Sup_fare else 0, axis=1)

#Tratamiento de Outliers columna tip_amount
Q1 = df_Impar["tip_amount"].quantile(0.01)
Q3 = df_Impar["tip_amount"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_Sup_tip = (Q3 + (1.5*IQR)) 
df_Impar["tip_amount_out"] = df_Impar.apply(lambda row: 1 if row["tip_amount"]>outliers_Sup_tip else 0, axis=1)

#se marco tip amount, ahora le toca a total amount

#Tratamiento de Outliers columna total_amount
Q1 = df_Impar["total_amount"].quantile(0.01)
Q3 = df_Impar["total_amount"].quantile(0.99)
#Se calcula el rango intercuartilico IQR.
IQR = Q3 -Q1
outliers_Sup_total_a = (Q3 + (1.5*IQR)) 
df_Impar["total_amount_out"] = df_Impar.apply(lambda row: 1 if row["tip_amount"]>outliers_Sup_total_a else 0, axis=1)


columna que marca si algun valor es un outlier

In [14]:
df_Impar["outlier"] = df_Impar.apply(lambda row: 1 if row["trip_distance_out"]==1 or row["fare_amount_out"]==1 or
row["total_amount_out"]==1 or row["tip_amount_out"]==1 else 0, axis=1)

fin del tratamiento de datos

In [15]:
df_Impar.to_parquet('C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/data/df_Impar_marcado.parquet',engine="pyarrow")

In [16]:
df_Impar = pd.read_parquet("C:/Users/andre/OneDrive/Escritorio/proyecto final 3/Proyecto-Taxis-NYC/data/df_Impar_marcado.parquet", engine="pyarrow")

In [17]:
df_Impar.head(1)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dayofmonth,hora,PUBorough,DOLocation,temp,feelslike,precip,snow,snowdepth,conditions,trip_distance_out,fare_amount_out,tip_amount_out,total_amount_out,outlier
0,2,2018-01-03,2018-01-03,1,4.75,1,88,230,2,21.0,0.0,0.5,0.0,0.0,0.3,22.0,3,2018-01-03,Manhattan,Manhattan,-7.3,-14.0,0.0,0.0,0.33,Clear,0,0,0,0,0


vendor id- ratecode id esta- payment_type.

revisar si agregar mas columnas al tratamiento de outliers.